# Scripts for reproducibility of ATAC analysis

# Figures 3 and S11

In [26]:
import scanpy as sc
import pandas as pd
import numpy as np

In [27]:
import os
import rpy2
#import anndata2ri

os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Resources'
os.environ['R_USER'] = '/Library/Frameworks/R.framework/Resources'

import anndata2ri

# anndata2ri interconverts AnnData and Single Cell Experiment objects
anndata2ri.activate()
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
%%R
library(ArchR)
library(rhdf5)
library(biomaRt)
library(dittoSeq)
library(dplyr)
library(Seurat)
library(RColorBrewer)
library(stringr)
library(parallel)
library(Seurat)
library(reticulate)
library(BSgenome.Mmusculus.UCSC.mm10)
library(ggplot2)
library(gplots)
library(ComplexHeatmap)
set.seed(16)

## Step1: Read ArchR object (download from Zenodo)

## Step2: Read "HSC_multiome_cleanup_lognorm_083123.h5ad" file (to retrieve cell barcodes and adata.obs columns of interest)

## Step 3: Match RNA_adata cell barcodes to ArchR object to create scatac.object.subset

In [ ]:
%%R -w 600 -h 600
Harmony_age <- plotEmbedding(ArchRProj = scatac.object.subset, colorBy = "cellColData", name = "age", embedding = "HARMONY_UMAP",baseSize = 15, labelSize=5,legendSize=5)

p<- Harmony_age + theme(panel.border = element_blank(),legend.text = element_text(size=15)) + geom_point(size = 1, stroke = 0.5) + scale_color_manual(values = c('#8FD694','#351431'))

#ggsave(paste0(outbase, 'ArchR_age.pdf'), p, dpi = 300)
p

In [ ]:
%%R -w 600 -h 600
Harmony_Fraticelli <- plotEmbedding(ArchRProj = scatac.object.subset, colorBy = "cellColData", name = "HSC_subsets", embedding = "HARMONY_UMAP",baseSize = 15, labelSize=5,legendSize=5)

p<- Harmony_Fraticelli + theme(panel.border = element_blank(),legend.text = element_text(size=15)) + geom_point(size = 0.5)+ scale_color_manual(values = c('#F6222E', '#16FF32', '#3283FE', '#FEAF16', '#D3D3D3'))

#ggsave(paste0(outbase, 'ArchR_cell_type_subset.pdf'), p, dpi = 300)
p

In [ ]:
%%R -w 600 -h 600
Clusters <- plotEmbedding(
    ArchRProj = scatac.object.subset,
    embedding = "HARMONY_UMAP",
    colorBy = "cellColData",
    name = c("Clusters"), baseSize = 15, labelSize=5,legendSize=5) + geom_point(size = 0.5) + theme(panel.border = element_blank(),legend.text = element_text(size=15))

#ggsave(paste0(outbase, 'ArchR_Seurat_Clusters.pdf'), Clusters, dpi = 300)
Clusters

## Step 4: Create Confusion Matrix

In [ ]:
cM <- confusionMatrix(paste0(scatac.object.subset$Clusters), paste0(scatac.object.subset$lowhigh))
library(pheatmap)

cM <- cM / Matrix::rowSums(cM)
colors <- brewer.pal(9, "YlOrRd")

In [ ]:
%%R -w 600 -h 600
p <- pheatmap::pheatmap(
    mat = as.matrix(cM), 
    color = colors, 
    border_color = "black", 
    fontsize = 15,          # Main font size, also affects the legend
    fontsize_row = 15,      # Row names font size
    fontsize_col = 15 
)
#ggsave(paste0(outbase, 'Cluster_vs_Kit_correlationmatrix.pdf'), p, dpi = 300)

%%R -w 600 -h 600
p1 <- pheatmap::pheatmap(
    mat = as.matrix(cM1), 
    color = colors, 
    border_color = "black", 
    fontsize = 15,          # Main font size, also affects the legend
    fontsize_row = 15,      # Row names font size
    fontsize_col = 15 
)
#ggsave(paste0(outbase, 'Cluster_vs_HSCsubset_correlationmatrix.pdf'), p1, dpi = 300)

## Step 5: Retrieve ArchR matrices to create an anndata object adata_atac for matrixplots

In [ ]:
%%R 
getAvailableMatrices(scatac.object.subset)

In [ ]:
%%R
# create a folder called "export" to save all the output
#save LSI result as csv 
# save any metadata for each cell as csv
# save the Harmony results:

In [ ]:
## Get genescore matrix and save as csv file 
# Peak counts
# Reorder peaks based on Chromosome order

In [ ]:
# Export counts
# create a subfolder called "peak_counts"
# get the peak counts
# save as a mtx file (Recall: mtx is a representation for saving sparse matrices efficiently)
# save cell names as csv
# save peak names as csv
# get Motif scores and save as csv 

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib_venn
from matplotlib.colors import LinearSegmentedColormap
import scanpy.external as sce
from matplotlib import cm, colors
from pathlib import Path
from bioinfokit import analys, visuz
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib.lines as lines

In [ ]:
adata_Chromvar = sc.AnnData(X =adata_atac.obsm['ChromVar'],
                       obs= adata_atac.obs,
                       obsm= adata_atac.obsm,
                       var=pd.DataFrame(index=adata_atac.uns['ChromVar_key']))

In [ ]:
adata_GeneScore = sc.AnnData(X =adata_atac.obsm['Gene_Scores'],
                       obs= adata_atac.obs,
                       obsm= adata_atac.obsm,
                       var=pd.DataFrame(index=adata_atac.uns['Gene_Scores_key']))

In [ ]:
var_names1 = ['Zbtb1_182','Zbtb7a_808', 'Runx1_712', 'Runx2_713', 'Runx3_858','Gata2_383', 'Gata3_384', 'Bcl11a_795', 'Bcl11b_814']

In [ ]:
lowhigh_chromvar = adata_Chromvar[(adata_Chromvar.obs['lowhigh'] == 'Low')|(adata_Chromvar.obs['lowhigh'] == 'High')]

sc.pl.matrixplot(
 lowhigh_chromvar,
    var_names= ['Zbtb7a_808', 'Runx1_712', 'Runx2_713', 'Runx3_858','Gata2_383', 'Gata3_384', 'Bcl11a_795', 'Bcl11b_814'], 
    groupby='lowhigh',
    colorbar_title="Motif-enrichment Score", cmap='RdBu_r',vmin=-0.25, vmax=0.25, 
    swap_axes=True, #save='DE_Chromvar_lowvshigh_TF_matrixplot.pdf')

In [ ]:
low_chromvar = adata_Chromvar[(adata_Chromvar.obs['agekit'] == 'young-Low')|(adata_Chromvar.obs['agekit'] == 'old-Low')]

sc.pl.matrixplot(
 low_chromvar,
    var_names=var_names1,
    groupby='agekit',
    colorbar_title="Motif-enrichment Score",cmap='RdBu_r',vmin=-0.25, vmax=0.25, 
    swap_axes=True, #save='Chromvar_low_oldvsyoung_matrixplot.pdf')

In [ ]:
var_names2 = ['Zbtb1', 'Zbtb7a', 'Ezh1','Ezh2', 'Runx1', 'Runx2', 'Runx3', 'Bcl11a']
adata_GeneScore.layers["scaled"] = sc.pp.scale(adata_GeneScore, copy=True).X

In [ ]:
cluster_test3  = adata_GeneScore[(adata_GeneScore.obs['lowhigh'] == 'Low') | (adata_GeneScore.obs['lowhigh'] == 'High')]
low = adata_GeneScore[(adata_GeneScore.obs['agekit'] == 'young-Low')|(adata_GeneScore.obs['agekit'] == 'old-Low')]

sc.pl.matrixplot(
 cluster_test3,
    var_names=var_names2,
    groupby='lowhigh', layer='scaled',
   cmap= 'RdBu_r', 
    swap_axes=True,
    colorbar_title='Gene Accessibility scores',#save='GeneAccessibilityScore_lowvshigh_matrixplot.pdf' )


sc.pl.matrixplot(
 low,
    var_names=['Zbtb1', 'Zbtb7a', 'Ezh1'],
    groupby='agekit',layer='scaled', vmin=-0.1, vmax=0.1,
   cmap= 'RdBu_r', 
    swap_axes=True,
    colorbar_title='Gene Accessibility scores', #save='GeneAccessibilityScore_low_oldvsyoung_matrixplot.pdf')